# Uninformed Search

This is the most basic form of seach.It implies trying to reach a goal state with no knowledge of where such goal state is.

Threfore, every possible strategy will just move from one node to another and check whether we reached the goal state or not. It will keep expanding until we reach the goal state.

We have an intrinsic measure of distance (or better, a *cost*) within a graph of nodes and paths, even if the absence of knowledge regarding the gold state: the number of steps associated to a given path. We can also consider adding other measure of distance given for example by the number of kilometers between the start state and our current state.

Given a graph with a cost defined for the paths, we can imagine 3 ways of searching.

- **Breadth-First Search (BFS)**: We start out with the root node and then expand the so-called frotier of examined nodes by considering other nodes that have the same *depth*. Here, the same depth means that the distance between the root node and each of the next-to-consider nodes is the same. In terms of the picture of a tree, this implies considering first expansions of the same depth. When there are ties (i.e. two nodes are one step away from current node) we can just pick one at random or use some rule of thumb.

- **Depth-First Search (DFS)**: We expand the *deepest* node in the current frontier of the tree. This means that the search proceeds to the deepest level of the current node, up to the point where nodes have no successors. As those nodes are exanded, they are removed from the frontier, so then the algorithm backs up to the next deepest node. 

- **Uniform Cost Seach**: Here the edges have a custom-defined cost and the expansion takes this into account, by expanding first the paths with the lowest cost $g(n)$. In fact, BFS can be considered as an optimal uniform cost search algorithm when all the costs are equal (the cost of a path is then directly proportional to the number of steps in the path).


![BFSvsDFS](./DFSBFS.png)

The implementation of Uninformed Search in pseudocode is a follows:


```function Graph.Search(problem):
  frontier = {[initial]}; explored={}
  loop:
    if frontier is empty: return FAIL
    path = remove.choice(frontier)
    s = path.end; add s to explored
    if s is a goal: return path
    for a in actions:
        add[path + a -> Result(s,a)]
        to frontier
        unless Result(s,a) in frontier or explored
```

So we first have a frontier initialized with with the root node as the only node. Then, we remove a path from the frontier and consider the state `s` that the removed path leads to. Then we add `s` to the explored list. If `s` is the goal state, we return the path. If not, we add to the frontier all the paths that can be constructed starting from `s` and adding each and all of the possible actions `a` (we do this unless `s+a` is in the explored list or is already in the frontier).

The difference between BFS and DFS is simply the path that is removed from the frontier at each iteration, i.e. the particular function or routine that we apply the line `path = remove.choice(frontier)` above. 


## Graph

In [4]:
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.children = []
        
    def add_child(self, new_node):
        self.children.append(new_node)
    
    def remove_child(self, del_node):
        if del_node in self.children:
            self.children.remove(del_node)

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list
        
    def add_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.add_child(node2)
            node2.add_child(node1)
            
    def remove_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.remove_child(node2)
            node2.remove_child(node1)

In [224]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

In [6]:
# To verify that the graph is created accurately.
# Let's just print all the parent nodes and child nodes.
for each in graph1.nodes:
    print('parent node = ',each.value,end='\nchildren\n')
    for each in each.children:
        print(each.value,end=' ')
    print('\n')

parent node =  S
children
R 

parent node =  H
children
G P 

parent node =  G
children
R A H 

parent node =  P
children
R H 

parent node =  R
children
G A P S 

parent node =  A
children
R G 



## Breadth-First Search

For this algorithm, the frontier is better to be represented with a FIFO (first-in-first-out) queue, which in python can be a usual queue or even a list

In [16]:
from queue import Queue
namequeue = Queue()
# Add elements
namequeue.put("Alice")
namequeue.put("Bob")
namequeue.put("Charlie")

# Remove elements
print(namequeue.get())
print(namequeue.get())
print(namequeue.get())

Alice
Bob
Charlie


In [4]:
namelist = []
namelist.append("Charlie")
namelist.append("Bob")
namelist.append("Alice")

# Remove elements
print(namelist.pop())
print(namelist.pop())
print(namelist.pop())

Alice
Bob
Charlie


Now we moving through the above graph using BFT. Implement the `bfs_search` to return the `GraphNode` with the value `search_value` starting at the `root_node`.

In [18]:
def bfs_search(root_node, search_value):
    bfs_frontier = Queue()
    bfs_frontier.put(root_node)
    bfs_explored = {}

    while True:
        if bfs_frontier.empty():
            return False

        # get (shallowest) node from frontier
        # ensured to be shallowest as frontier is FIFO
        current_node = bfs_frontier.get()
        # note: we don't check gold state here!
        bfs_explored.add(current_node)

        # consider the result of each possible action on 
        # the current node
        for next_node in current_node.children:
            # bfs_frontier is not iterable but bfs_frontier.queue is
            if (
                (next_node not in bfs_explored) and (next_node not in bfs_frontier.queue) 
                ):
                if next_node.value == search_value:
                    # return child node if condition is satisfied
                    # next_node is goal
                    return next_node
                # add child nodes to the frontier
                bfs_frontier.put(next_node)


In [19]:
assert nodeA == bfs_search(nodeS, 'A')
assert nodeS == bfs_search(nodeP, 'S')
assert nodeR == bfs_search(nodeH, 'R')

### Faster solution (Udacity)

In [ ]:
# Solution
def bfs_search(root_node, search_value):
    visited = set()                           # Sets are faster while lookup. Lists are faster to iterate.
    queue = [root_node]
    
    while len(queue) > 0:
        current_node = queue.pop(0)
        visited.add(current_node)

        if current_node.value == search_value:
            return current_node

        for child in current_node.children:
            if child not in visited:          # Lookup
                queue.append(child)


## Depth-First search

For this type of search a LIFO (last-in-first-out) queue is used to model the frontier, as the latest node to be incorporated is also the first one wose children will be considered.

In [1]:
import queue
lifonamequeue = queue.LifoQueue()
# Add elements
lifonamequeue.put("Alice")
lifonamequeue.put("Bob")
lifonamequeue.put("Charlie")

# Remove elements
print(lifonamequeue.get())
print(lifonamequeue.get())
print(lifonamequeue.get())


Charlie
Bob
Alice


We see that the last element to be put is the first one out. In this sense, it goes contrary to a LIFO queue.

In [241]:
def dfs_search_recursion(start_node, search_value, visited_nodes = set(), lifo_nodes=[]):
    # start_node works also as current node
    # finish if the find the goal
    if start_node.value == search_value:
        return start_node

    # add current node to visited nodes
    visited_nodes.add(start_node)
    # add all the non-visited children of the current node
    # (located at the end of the LIFO queue!)
    lifo_nodes.extend(list(set(start_node.children) - set(visited_nodes)))
    

    while len(lifo_nodes) > 0:
        # extract the last node from the LIFO queue
        # this will be one of the latest added nodes
        child = lifo_nodes.pop(-1)
        # process most-recently added children first: DFS
        if child not in visited_nodes: # redundant in principle: kept just in case
            return dfs_search_recursion(child, search_value, visited_nodes, lifo_nodes)

In [242]:
assert nodeA == dfs_search_recursion(nodeG, 'A', visited_nodes = set(), lifo_nodes=[])
assert nodeA == dfs_search_recursion(nodeS, 'A', visited_nodes = set(), lifo_nodes=[])
assert nodeS == dfs_search_recursion(nodeP, 'S', visited_nodes = set(), lifo_nodes=[])
assert nodeR == dfs_search_recursion(nodeH, 'R', visited_nodes = set(), lifo_nodes=[])

In [243]:
for i in range(100000):
    if i % 10000 == 0:
        print(i)
    if dfs_search_recursion(nodeG, 'A', visited_nodes = set(), lifo_nodes=[]).value  != 'A':
        print('errr')
        break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


### Udacity's solution (WRONG!)

In [233]:
# Solution
def dfs_recursion_start(start_node, search_value):
    visited = set()               # Set to keep track of visited nodes.
    return dfs_recursion(start_node, visited, search_value)

# Recursive function
def dfs_recursion(node, visited, search_value):
    if node.value == search_value:
        found = True              # Don't search in other branches, if found = True
        return node
    
    visited.add(node)
    found = False
    result = None

    # Conditional recurse on each neighbour
    for child in node.children:
        if (child not in visited):
                result = dfs_recursion(child, visited, search_value)
                
                # Once the match is found, no more recurse 
                if found:
                    break
    return result

Hi,

I have noticed that if I remove the direct connection `G <-> A` in the graph defined in the exercise notebook of the DFS algorithm I get an assertion error in the following line:

`assert nodeA == dfs_recursion_start(nodeG, 'A')`

However, I would expect the DFS algorithm to still find the goal node through the path `G -> R -> A`, so there should be some issue with the proposed solution.

Assuming that I am not missing anything, I believe that the problem has to do with the fact that reassignment

`found = True`

has only the scope of the current node, and not on the previous node in the recursive function. Therefore, the break condition within the loop is not always triggered when it should.

I think that the current solution will crash whenever the goal node is a leaf node that is not directly connected with start_node, and the reason why it works for the graph currently defined in the exercise if because there is an interplay between visited nodes and connections so that the goal node happens to always be the last node to be checked.

Is there anything that I am not interpreting correctly?

Thanks

In [235]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
#graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

assert nodeA == dfs_recursion_start(nodeG, 'A')
assert nodeA == dfs_recursion_start(nodeS, 'A')
assert nodeS == dfs_recursion_start(nodeP, 'S')
assert nodeR == dfs_recursion_start(nodeH, 'R')

AssertionError: 

### Correcting Udacity's solution

Basically we can write a workaround for Udacity's solution while keeping as much as the original code as possible by simply ensuring that variable `found` is propagated as expected.

In [236]:
# Solution
def dfs_recursion_start(start_node, search_value):
    visited = set()               # Set to keep track of visited nodes.
    return dfs_recursion(start_node, visited, search_value)

# Recursive function
def dfs_recursion(node, visited, search_value):
    if node.value == search_value:
        return node, True
    visited.add(node)
    found = False
    result = None

    # Conditional recurse on each neighbour
    for child in node.children:
        if (child not in visited):
                result, found = dfs_recursion(child, visited, search_value)
                # Once the match is found, no more recurse 
                if found:
                    break
    return result, found

In [240]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
#graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

assert nodeA == dfs_recursion_start(nodeG, 'A')[0]
assert nodeA == dfs_recursion_start(nodeS, 'A')[0]
assert nodeS == dfs_recursion_start(nodeP, 'S')[0]
assert nodeR == dfs_recursion_start(nodeH, 'R')[0]

## Uniform Cost Search (Dijkstra's Algorithm)

For this type of algorithm the frontier can be modelled as a queue in which the nodes are stored in order according to the cost $g(n)$


<img style="float: center;height:250px;" src="graph_dijs.png"><br>

#### The Algorithm
1. Create a `result` dictionary. At the end of the program, `result` will have the shortest distance (value) for all nodes (key) in the graph. For our example, it will become as `{'A': 0, 'B': 5, 'C': 3, 'D': 2, 'F': 6, 'E': 4}`<br><br>
1. Start with the source node. Distance from source to source itself is 0.  <br><br>
1. The distance to all other nodes from the source is unknown initially, therefore set the initial distance to infinity.  <br><br>
1. Create a set `unvisited` containing nodes that have not been visited. Initially, it will have all nodes of the graph.<br><br>
1. Create a `path` dictionary that keeps track of the previous node (value) that can lead to the current node (key) in an optimal way. At the end of the program, for our example, it will become as `{'B': 'A', 'C': 'D', 'D': 'A', 'F': 'C', 'E': 'C'}`. Values represent the **Previous Optimal Node**. Here, the Previous Optimal Node is the "best" node which could lead us to the current node (key). For instance, the previous optimal node of `'C'` is `'D'` because the optimal path (in term of distance) that starts at `'A'` and finishes at `'C'` passes through `'D'` before reaching `'C'`. In this sense, the optimal path between the source node (`'A'` in this case) and any of the other nodes can be found using `path`: start at the key given by the goal node and move backwards according to dictionary `path`. For example, If we want to know the optimal path from `'A'` to `'E'`, we note that `path['E']='C'`, then `path['C']='D'` and finally `path['D']='A'`, so the optimal path from `'A'` to `'E'` would be `'A' -> 'D' -> 'C' -> 'E' `. This dictionary `path` can be initialized as an empty one. <br><br>
1. As long as `unvisited` is non-empty, repeat the following:
 - Find the unvisited node having smallest known distance from the *source node*. This defines the *current node* <br><br>
 - For the *current node*, find all the **unvisited neighbours** of it. Then calculate the distance of each unvisited neighbour to the *source node*.  <br><br>
 - If the calculated distance of the **unvisited neighbour** is less than the already known distance in `result` dictionary, update the shortest distance in the `result` dictionary. <br><br>
 - If there is an update in the `result` dictionary, you need to update the `path` dictionary as well for the same key. The update entails placing value = *current node*, being the key the neighbour of *current node* for which the distance in `result` has been updated. Basically, we are placing the new previous optimal node. <br><br>
 - Remove the current node from the `unvisited` set.

Note that this (greedy) algorithm ensures finding an optimal solution for any given starting or *source node* because it looks for the minimum distance expanding from nodes that are the closest to the source at each time. The fact that the expansion is done according to distance (hence in an ordered way) allows the algorithm to be only $\mathcal{O}(n^2)$: for each of the closest nodes to the source ($\mathcal{O}(n)$) we search through all its neighbours ($\mathcal{O}(n)$). So the search is actually done in triplets:

$$ \text{source node} \rightarrow \text{current node} \rightarrow \text{current-node's neighbour} $$

but this is OK because, as we expand in an ordered way, no matter how far a node is from the source, *current node* will always be closer to *source node* than its (unvisited) neighbours. For instance, imagine that there are two possible paths to reach node `'E'` from the source node `'A'`:

$$ A \rightarrow F \rightarrow D \rightarrow E  $$
$$ A \rightarrow B \rightarrow C \rightarrow E  $$

Both `'C'` and `'D'` are closer to `'A'` than `'E'` and this is crucial because this means that the algorithm will process them before processing `'E'`. Imagine that the optimal path from `'A'` to `'E'` passes through `'D'` but `'C'` is closer to `'A'` than `'D'`. Then, we will first process the triplet 

$$ \text{source node} (A) \rightarrow \text{current node} (C) \rightarrow \text{current-node's neighbour} (E) $$

and note that we can skip $B$ because it is not necessary, as we already know that the optimal path to $C$ passes through $B$. That is why we can express everything in terms of triplets only. This will give a value to `result` and `path`. But then, we will process 

$$ \text{source node} (A) \rightarrow \text{current node} (D) \rightarrow \text{current-node's neighbour} (E) $$

(again, `'F'` can be safely skipped as optimal path to `'D'` passes through `'F'` but this is not important). Si we have assumed that this path is optimal, values of `result` and `path` will be updated accordingly, reflecting this.

So, crucially, optimal paths to a given node $N$ can be found in a Markovian way if we start from all the optimal paths to previous nodes whose neighbour is $N$ and search for the optimal 'step ahead'.

**Note** - This implementation of the Dijkstra's algorithm is not very efficient. Currently it has a $\mathcal{O}(n^2)$  time complexity. We will see a better version in the next lesson - "Graph Algorithms" with $\mathcal{O}(n\log(n))$  time complexity.

In [1]:
# Helper Code - define graph with distance
from collections import defaultdict
class Graph:
    def __init__(self):
        self.nodes = set()                   # A set cannot contain duplicate nodes
        self.neighbours = defaultdict(list)  # Defaultdict is a child class of Dictionary that provides a default value for a key that does not exists.
        self.distances = {}                  # Dictionary. An example record as ('A', 'B'): 6 shows the distance between 'A' to 'B' is 6 units

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        self.neighbours[from_node].append(to_node)
        self.neighbours[to_node].append(from_node)
        self.distances[(from_node, to_node)] = distance
        self.distances[(to_node, from_node)] = distance    # lets make the graph undirected / bidirectional 
        
    def print_graph(self):
        print("Set of Nodes are: ", self.nodes)
        print("Neighbours are: ", self.neighbours)
        print("Distances are: ", self.distances)

In [2]:
def dijkstra(graph, source):
    # Declare and initialize result, unvisited, and path
    # result[source node] = 0, impying that the first nodes to be 
    # considered will be the neighbours of the source node. 
    # (in other words current_node = source_node in the first iteration)
    result = {node_name: 1E10 if node_name != source else 0. for node_name in graph.nodes}
    unvisited = graph.nodes.copy()
    path = {}
    # As long as unvisited is non-empty
    while unvisited:
        current_node = None
        # 1. Find the unvisited node having smallest known distance from the source node.
        # (NO MATTER IF THIS NODE IS DIRECTLY CONNECTED OR NOT WITH THE SOURCE,
        # distance here does not mean elements in graph.distances, but total distances)
        for elm in [pair for pair in dict(sorted(result.items(), key=lambda x: x[1])).keys()]:
            # process first the nodes closer to the source
            if elm in unvisited:
                current_node = elm
                break
        # stop when there is no solution
        if current_node is None:
            break
        
        # 2. For the current node, find all the unvisited neighbours of it. 
        # Then (re)calculate the distance of each unvisited neighbour to the source node
        # assuming that we pass through the current node.
        unvisited_neights_current = set(graph.neighbours[current_node]) & unvisited
        distance_unv_neigths = {
            i: result[current_node] + graph.distances[(current_node, i)]
            for i in unvisited_neights_current
            }
        # 3. If the above calculated distance of the unvisited neighbour to the source
        # is less than the already known distance in result dictionary, 
        # update the shortest distance in the result dictionary.        
        updated_dist =  {
            node_neigh : dist for node_neigh, dist in distance_unv_neigths.items()
            if dist < result[node_neigh]
            }
        #update result
        result = {**result, **updated_dist}
        # 4. If there is an update in the result dictionary, you need to update the
        # path dictionary as well for the same key.
        if updated_dist:
            updated_path = {
                node_neigh : current_node for node_neigh in updated_dist.keys()
            }
            path = {**path, **updated_path}
        # 5. Remove the current node from the unvisited set.
        unvisited.remove(current_node)
    return result

In [9]:
# Test 1
testGraph = Graph()
for node in ['A', 'B', 'C', 'D', 'E']:
    testGraph.add_node(node)

testGraph.add_edge('A','B',3)
testGraph.add_edge('A','D',2)
testGraph.add_edge('B','D',4)
testGraph.add_edge('B','E',6)
testGraph.add_edge('B','C',1)
testGraph.add_edge('C','E',2)
testGraph.add_edge('E','D',1)

print(dijkstra(testGraph, 'A'))     # {'A': 0, 'D': 2, 'B': 3, 'E': 3, 'C': 4}

# Test 2
graph = Graph()
for node in ['A', 'B', 'C']:
    graph.add_node(node)
    
graph.add_edge('A', 'B', 5)
graph.add_edge('B', 'C', 5)
graph.add_edge('A', 'C', 10)

print(dijkstra(graph, 'A'))        # {'A': 0, 'C': 10, 'B': 5}

# Test 3
graph = Graph()
for node in ['A', 'B', 'C', 'D', 'E', 'F']:
    graph.add_node(node)
    
graph.add_edge('A', 'B', 5)
graph.add_edge('A', 'C', 4)
graph.add_edge('D', 'C', 1)
graph.add_edge('B', 'C', 2)
graph.add_edge('A', 'D', 2)
graph.add_edge('B', 'F', 2)
graph.add_edge('C', 'F', 3)
graph.add_edge('E', 'F', 2)
graph.add_edge('C', 'E', 1)

print(dijkstra(graph, 'A'))       # {'A': 0, 'C': 3, 'B': 5, 'E': 4, 'D': 2, 'F': 6}

{'B': 3.0, 'D': 2.0, 'E': 3.0, 'C': 4.0, 'A': 0.0}
{'C': 10.0, 'A': 0.0, 'B': 5.0}
{'B': 5.0, 'D': 2.0, 'E': 4.0, 'C': 3.0, 'A': 0.0, 'F': 6.0}


### Udacity's solution (very similar in this case)

In [4]:
import sys

'''Find the shortest path from the source node to every other node in the given graph'''
def dijkstra_ud(graph, source):
    
    result = {}
    result[source] = 0                 
    
    for node in graph.nodes:
        if (node != source):
            result[node] = sys.maxsize
            
    unvisited = set(graph.nodes)  
    
    path = {}

    '''THE GREEDY APPROACH'''
    # As long as unvisited is non-empty
    while unvisited: 
        min_node = None    
        
        # 1. Find the unvisited node having smallest known distance from the source node.
        for node in unvisited:
            if node in result:
                
                if min_node is None:       
                    min_node = node
                elif result[node] < result[min_node]:
                    min_node = node

        if min_node is None:
            break
            
        # known distance of min_node
        current_distance = result[min_node]
        
        # 2. For the current node, find all the unvisited neighbours. 
        # For this, you have calculate the distance of each unvisited neighbour.
        for neighbour in graph.neighbours[min_node]:
            if neighbour in unvisited:
                distance = current_distance + graph.distances[(min_node, neighbour)]
            
                # 3. If the calculated distance of the unvisited neighbour is less than the already known distance in result dictionary, update the shortest distance in the result dictionary.
                if ((neighbour not in result) or (distance < result[neighbour])):
                    result[neighbour] = distance

                    # 4. If there is an update in the result dictionary, you need to update the path dictionary as well for the same key.
                    path[neighbour] = min_node
        
        # 5. Remove the current node from the unvisited set.
        unvisited.remove(min_node)

    return result

In [10]:
# Test 1
testGraph = Graph()
for node in ['A', 'B', 'C', 'D', 'E']:
    testGraph.add_node(node)

testGraph.add_edge('A','B',3)
testGraph.add_edge('A','D',2)
testGraph.add_edge('B','D',4)
testGraph.add_edge('B','E',6)
testGraph.add_edge('B','C',1)
testGraph.add_edge('C','E',2)
testGraph.add_edge('E','D',1)

print(dijkstra_ud(testGraph, 'A'))     # {'A': 0, 'D': 2, 'B': 3, 'E': 3, 'C': 4}

# Test 2
graph = Graph()
for node in ['A', 'B', 'C']:
    graph.add_node(node)
    
graph.add_edge('A', 'B', 5)
graph.add_edge('B', 'C', 5)
graph.add_edge('A', 'C', 10)

print(dijkstra_ud(graph, 'A'))        # {'A': 0, 'C': 10, 'B': 5}

# Test 3
graph = Graph()
for node in ['A', 'B', 'C', 'D', 'E', 'F']:
    graph.add_node(node)
    
graph.add_edge('A', 'B', 5)
graph.add_edge('A', 'C', 4)
graph.add_edge('D', 'C', 1)
graph.add_edge('B', 'C', 2)
graph.add_edge('A', 'D', 2)
graph.add_edge('B', 'F', 2)
graph.add_edge('C', 'F', 3)
graph.add_edge('E', 'F', 2)
graph.add_edge('C', 'E', 1)

print(dijkstra_ud(graph, 'A'))       # {'A': 0, 'C': 3, 'B': 5, 'E': 4, 'D': 2, 'F': 6}

{'A': 0, 'B': 3, 'D': 2, 'E': 3, 'C': 4}
{'A': 0, 'C': 10, 'B': 5}
{'A': 0, 'B': 5, 'D': 2, 'E': 4, 'C': 3, 'F': 6}
